In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_probability as tfp
import numpy as np
from matplotlib import pyplot as plt
import tqdm
from scipy.io import savemat
import cv2
from scipy.special import factorial
import itertools
from numpy.linalg import norm as np_norm

import sys
sys.path.append('../dependencies/')
import dataset_utils
import network_ec_bm as network
import utils
import zca

import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)


def rotate(img, angle):
    rotated_img = np.zeros(img.shape)
    if angle == 90:
        rot_func = cv2.cv2.ROTATE_90_CLOCKWISE
    elif angle == 180:
        rot_func = cv2.cv2.ROTATE_180
    elif angle == 270:
        rot_func = cv2.ROTATE_90_COUNTERCLOCKWISE
    for i in range (img.shape[0]):
        temp = cv2.rotate(img[i,:,:,:], rot_func)
        if len(temp.shape) == 2:
            temp = np.expand_dims(temp, axis=-1)
        rotated_img[i,:,:,:] = temp
    return np.round(rotated_img)

def rotate_inv(img, angle):
    rotated_img = np.zeros(img.shape)
    if angle == 90:
        rot_func = cv2.cv2.ROTATE_90_CLOCKWISE
    elif angle == 180:
        rot_func = cv2.cv2.ROTATE_180
    elif angle == 270:
        rot_func = cv2.ROTATE_90_COUNTERCLOCKWISE
    for i in range (img.shape[0]):
        temp = cv2.rotate(cv2.flip(img[i,:,:,:],1), rot_func)
        if len(temp.shape) == 2:
            temp = np.expand_dims(temp, axis=-1)
        rotated_img[i,:,:,:] = temp
    return np.round(rotated_img)

def inv(img):
    inverted_img = np.zeros(img.shape)
    for i in range (img.shape[0]):
        temp = cv2.flip(img[i,:,:,:],1)
        if len(temp.shape) == 2:
            temp = np.expand_dims(temp, axis=-1)
        inverted_img[i,:,:,:] = temp
    return np.round(inverted_img)

def derangement(n):
    orders = np.array(list(itertools.permutations(np.arange(n*n))))
    tmp = []
    for i in range (orders.shape[0]):
        count = 0
        for j in range (orders.shape[1]):
            if orders[i,j] == j:
                count += 1
        if count == 0:
            tmp.append(orders[i,:])
    return np.array(tmp), np.array(tmp).shape[0]

def permutate(n):
    x = np.array(list(itertools.permutations(range(0, n*n))))
    x = np.delete(x, 0, 0)
    for i in range (n*n):
        x = np.concatenate(( np.concatenate((x, np.zeros((x.shape[0], 1))), axis=1),
                             np.concatenate((x, np.ones((x.shape[0], 1))), axis=1)),
                             axis=0)
    for i in range (n*n):
        x = np.concatenate(( np.concatenate((x, np.zeros((x.shape[0], 1))), axis=1),
                             np.concatenate((x, np.ones((x.shape[0], 1))), axis=1),
                             np.concatenate((x, 2*np.ones((x.shape[0], 1))), axis=1)),
                             axis=0)
    return x.astype('int32')

def rotate_config(imgs, angle, inverse):
    if inverse == 1:
        imgs = inv(imgs)
    if angle == 0:
        imgs = rotate(imgs, 90)
    elif angle == 1:
        imgs = rotate(imgs, 180)
    elif angle == 2:
        imgs = rotate(imgs, 270)
    return imgs
        
def patch_shuffle(imgs, n, orders, shuffles):
    
    shap = imgs.shape
    patches = np.zeros((shap[0], int(shap[1]/n), int(shap[2]/n), shap[3], int(n*n)))
    patch_shap = patches.shape
    shuffled = np.zeros((shap[0], shap[1], shap[2], shap[3], orders.shape[0]))
            
    for k in range (orders.shape[0]):
        temp = imgs.copy()
        for i in range (n):
            for j in range (n):
                patches[:,:,:,:,i*n+j] = temp[:, i*(patch_shap[1]):(i+1)*(patch_shap[1]),
                                              j*(patch_shap[2]):(j+1)*(patch_shap[2]), :]
        for i in range (n*n):
            patches[:,:,:,:,i] = rotate_config(patches[:,:,:,:,i], np.squeeze(orders[k,2*n*n+i]),
                                               np.squeeze(orders[k,n*n+i]))
        for i in range (n):
            for j in range (n):
                shuffled[:, i*(patch_shap[1]):(i+1)*(patch_shap[1]), j*(patch_shap[2]):(j+1)*(patch_shap[2])
                         , :, k] = patches[:,:,:,:,np.squeeze(orders[k,i*n+j])]
    return shuffled
    
def dissimilarity(a,b):
    cos = np.zeros(a.shape[0])
    for i in range (a.shape[0]):
        cos[i] = (1 - (np.dot(a[i,:,:,:].flatten(), b[i,:,:,:].flatten())/(np_norm(a[i,:,:,:].flatten())*np_norm(b[i,:,:,:].flatten()))))/2
    return cos

tf.random.set_seed(42)

tfd = tfp.distributions
tfk = tf.keras
tfkl = tf.keras.layers

train_set = 'sign_lang'
norm = None
input_shape = (32,32,3)
n_patches = 2
orders = permutate(n_patches)
shuffles = 20
samples = np.random.randint(0, high=orders.shape[0], size=shuffles)
orders = orders[samples,:]
mode = 'grayscale'
pre = 'ec_bm/'

if mode == 'color':
    input_shape = (32, 32, 3)
    datasets = [
        'svhn_cropped',
        'cifar10',
        'celeb_a',
        'gtsrb',
        'compcars',
        'noise'
    ]
    num_filters = 64
    batch_size = 512
elif mode == 'grayscale':
    input_shape = (32, 32, 1)
    datasets = [
        'mnist',
        'fashion_mnist',
        'emnist/letters',
        'sign_lang',
        'noise'
    ]
    num_filters = 32
    batch_size = 2048

reg_weight = 0
num_resnet = 2
num_hierarchies = 4
num_logistic_mix = 5
num_filters = num_filters
dropout_p = 0.3
learning_rate = 1e-3
use_weight_norm = True
epochs = 100
optimizer = tf.optimizers.Adam(learning_rate=learning_rate)
    
if norm is None:
    dir_str = 'original'
elif norm == 'pctile-5':
    dir_str = 'pctile-5'
elif norm == 'channelwhiten':
    dir_str = 'zca'
elif norm == 'zca_original':
    dir_str = 'zca_original'
elif norm == 'histeq':
    dir_str = 'histeq'
    
model_dir = '../saved_models/' + pre + dir_str + '/' + train_set + '/'
if not os.path.exists(model_dir):
    os.makedirs(model_dir)
    
if norm == 'zca_original':
    zca_transform = zca.compute_zca(train_set)
else:
    zca_transform = None

dist = network.PixelCNN(
      image_shape=input_shape,
      num_resnet=num_resnet,
      num_hierarchies=num_hierarchies,
      num_filters=num_filters,
      num_logistic_mix=num_logistic_mix,
      dropout_p=dropout_p,
      use_weight_norm=use_weight_norm,
)

image_input = tfkl.Input(shape=input_shape)
log_prob = dist.log_prob(image_input)
model = tfk.Model(inputs=image_input, outputs=log_prob)
model.add_loss(-tf.reduce_mean(log_prob))
model.compile(optimizer=optimizer)

model.build([None] + list(input_shape))
model.load_weights(model_dir+'weights')

probs = {}

for dataset in datasets:
    
    _, _, ds_test = dataset_utils.get_dataset(
          dataset,
          batch_size,
          mode,
          normalize=norm,
          dequantize=False,
          visible_dist='mixture_of_logistics',
          zca_transform=zca_transform,
          mutation_rate=0
      )
    
    tmp = []
    for i in range (shuffles):
        globals()['tmp_'+str(i)] = []

    for test_batch in tqdm.tqdm(ds_test):
        batch = tf.cast(test_batch, tf.float32).numpy()
        tmp.append(dist.log_prob(batch, training=False).numpy())
        patches = patch_shuffle(batch, n_patches, orders, shuffles)
        for i in range (shuffles):
            globals()['tmp_'+str(i)].append(dist.log_prob(np.round(patches[:,:,:,:,i]), training=False).numpy())

    tmp = np.expand_dims(np.concatenate(tmp, axis=0),axis=-1)

    probs[dataset+'_regular'] = tmp
    for i in range (shuffles):
        globals()['tmp_'+str(i)] = np.expand_dims(np.concatenate(globals()['tmp_'+str(i)], axis=0),axis=-1)

    for i in range (shuffles):
        probs[dataset+'_shuffle_'+str(i)] = globals()['tmp_'+str(i)]
        if i == 0:
            probs[dataset] = tmp - globals()['tmp_'+str(i)]
        else:
            probs[dataset] += tmp - globals()['tmp_'+str(i)]


if train_set == 'emnist/letters':
    train_set = 'emnist_letters'
    
dir_str1 = pre + dir_str + '_patch_inv_rot_' + str(shuffles) + '_shuffles'

save_dir = '../probs/' + dir_str1 + '/'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)
    
savemat(save_dir + train_set + '.mat', probs)

2022-05-09 23:37:57.753836: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-05-09 23:37:58.184531: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 10407 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1080 Ti, pci bus id: 0000:41:00.0, compute capability: 6.1
  0%|                                                                                                                                                                                      | 0/5 [00:00<?, ?it/s]2022-05-09 23:38:10.633103: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2022-05-09 23:38:11.173515:

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [03:58<00:00, 29.87s/it]
4it [02:02, 30.64s/it]
5it [02:31, 30.37s/it]
